In [1]:
!pip install -q transformers accelerate pandas tqdm huggingface_hub

from huggingface_hub import login
login(token="")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from tqdm import tqdm

tqdm.pandas()

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
from math import ceil
from tqdm import tqdm

bias_classes = ["group_2", "group_1", "no_bias"]

# Shared system prompt
system_prompt = (
    "You are a policy-analysis assistant.\n\n"
    "Your task is to classify each policy excerpt into **exactly one** of the following three categories, based on its main subject and social framing.\n\n"

    "Understanding Bias:\n"
    "- A policy has a bias if it distinguishes, benefits, restricts, or treats people differently based on identity, legal status, or economic position.\n"
    "- This includes both: when a policy provides support, protections, or special resources for a group and when a policy excludes, limits, penalizes, or places extra burdens on a group.\n"
    "- Even if the wording is neutral, if any group is mentioned, implied, or treated differently, the policy has a bias.\n\n"

    "Understanding No Bias:\n"
    "- A policy has no bias when it is entirely procedural, factual, or operational, and does not involve any group-based treatment.\n"
    "- No identity groups, legal status, or economic classifications should be mentioned or implied.\n"
    "- Examples include timelines, implementation steps, staff protocols, or technical logistics.\n\n"

    "Classification Categories:\n"
    "group_2 → Policies involving personal identity or socioeconomic status:\n"
    "   - economic: poverty, income level, wealth, rent support, financial hardship, housing aid.\n"
    "   - racial/cultural: race, ethnicity, cultural identity, minority status, racial disparities.\n"
    "   - age: elderly, children, youth-targeted policies, age-based restrictions or services.\n"
    "   - religion: religious practices, faith-based exceptions, worship protections.\n"
    "   - gender: women's rights, gender identity, gender discrimination, reproductive policy.\n\n"
    "group_1 → Policies focused on legal, civic, or institutional treatment:\n"
    "   - political: elections, campaigning, districting, war related, international realtions, veteran related.\n"
    "   - criminal justice: policing, sentencing, incarceration, parole, law enforcement.\n"
    "   - citizenship: immigration status, voting rights, naturalization, deportation, visa policy.\n"
    "   - disability: medical, accommodations, physical/mental impairments, special education services.\n"
    "   - education: student placement, curriculum, testing access, teaching, education access, language proficiency, school enrollment.\n\n"
    "no_bias → Neutral, operational, or internal content with no group implications:\n"
    "   - Meeting schedules, titles, definitions, procedural instructions, timelines, internal processes, technical steps.\n\n"

    "Classification Rules:\n"
    "- If a group is named, implied, or treated differently, assign a bias label (group_1 or group_2) based on the domain.\n"
     "- A policy has a bias if it distinguishes, benefits, restricts, or treats people differently based on identity, legal status, or economic position.\n"
    "- A policy has no bias when it is entirely procedural, factual, or operational, and does not involve any group-based treatment.\n"
    "- Do not default to any label. Treat all three categories as equally likely.\n"
    "- Pick the most specific match based on content.\n"
    "- Return only one label: `no_bias`, `group_1`, or `group_2` — all lowercase, no punctuation, no explanation."

    "FEW-SHOT EXAMPLES\n"
    "─────────────────\n"
    "[1]\nPolicy:\n“Applicants must have resided legally in the country for five consecutive years before they can petition for naturalization.”\nOutput → group_1\n\n"
    "[2]\nPolicy:\n“The act allocates additional funding for wheelchair-accessible entrances in all public elementary schools.”\nOutput → group_1\n\n"
    "[3]\nPolicy:\n“Households earning below 150% of the poverty line qualify for a monthly food-assistance credit.”\nOutput → group_2\n\n"
    "[4]\nPolicy:\n“Women over 40 will receive free annual breast-cancer screenings.”\nOutput → group_2\n\n"
    "[5]\nPolicy:\n“Native language instruction will be expanded for students with limited English proficiency.”\nOutput → group_1\n\n"
    "[6]\nPolicy:\n“Veterans will receive priority access to affordable housing in metropolitan areas.”\nOutput → group_2\n\n"
    "[7]\nPolicy:\n“The Commissioner shall publish the annual report within 30 days of fiscal year-end.”\nOutput → no_bias\n\n"
    "[8]\nPolicy:\n“The statute’s sunset clause to 2035.”\nOutput → no_bias"
)


def classify_batch(df, batch_size=32, column="policy_perturbed"):
    all_preds = []
    num_batches = ceil(len(df) / batch_size)

    for i in tqdm(range(num_batches), desc="Classifying in batches"):
        batch_df = df.iloc[i * batch_size : (i + 1) * batch_size]
        prompts = []

        for description in batch_df[column]:
            user_prompt = (
                f"Classify the following policy excerpt.\n"
                f"Return only one of: no_bias, group_1, or group_2\n\n"
                f"Excerpt:\n\"\"\"\n{description}\n\"\"\"\n\n"
                f"Your answer:"
            )

            full_prompt = tokenizer.apply_chat_template([
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ], tokenize=False, add_generation_prompt=True)

            prompts.append(full_prompt)

        # Run batched generation
        outputs = pipe(prompts, max_new_tokens=20, do_sample=False)

        for output, prompt in zip(outputs, prompts):
            generated = output[0]['generated_text']
            prediction = generated[len(prompt):].strip().split("\n")[0].lower()

            normalized_pred = (
                prediction.replace("_", "")
                          .replace("-", "")
                          .replace(".", "")
                          .replace("(", "")
                          .replace(")", "")
                          .strip()
            )

            label = "unknown"
            for cls in bias_classes:
                if normalized_pred == cls.replace("_", ""):
                    label = cls
                    break

            all_preds.append(label)

    return all_preds

In [6]:
import pandas as pd
from math import ceil
from tqdm import tqdm
import os
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')

# Define output directory (overwrite okay)
output_dir = "/content/drive/MyDrive/policy_bias_checkpoints"
os.makedirs(output_dir, exist_ok=True)

# Load full dataset
df = pd.read_csv("FINAL_PERTURBED_DATASET.csv")     

# Create an empty column to hold your running predictions
df['predicted_bias'] = None

batch_size  = 32                          
num_batches = ceil(len(df) / batch_size)

for batch_i in tqdm(range(num_batches), desc="Classifying and checkpointing"):
    start = batch_i * batch_size
    end   = min(start + batch_size, len(df))

    # slice out just this batch
    batch_idx = df.index[start:end]
    batch_df  = df.loc[batch_idx]

    # run your classifier on just this slice
    preds = classify_batch(batch_df, batch_size=batch_size)

    # write those back into your master DataFrame
    df.loc[batch_idx, 'predicted_bias'] = preds

    # now save the entire “so far” (or the full df) to a checkpoint
    ckpt_name = f"pert_checkpoint_up_to_{end}.csv"
    df.to_csv(os.path.join(output_dir, ckpt_name), index=False)
    print(f"  → saved full table with predictions through row {end} to {ckpt_name}")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Classifying in batches:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(

Classifying and checkpointing:   0%|          | 1/647 [00:08<1:34:11,  8.75s/it]

  → saved full table with predictions through row 32 to pert_checkpoint_up_to_32.csv



Classifying in batches:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(

Classifying and checkpointing:   0%|          | 2/647 [00:17<1:33:35,  8.71s/it]

  → saved full table with predictions through row 64 to pert_checkpoint_up_to_64.csv


...

  → saved full table with predictions through row 20640 to pert_checkpoint_up_to_20640.csv



Classifying in batches:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(

Classifying and checkpointing: 100%|█████████▉| 646/647 [1:08:53<00:06,  6.21s/it]

  → saved full table with predictions through row 20672 to pert_checkpoint_up_to_20672.csv



Classifying in batches:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(

Classifying and checkpointing: 100%|██████████| 647/647 [1:08:57<00:00,  6.40s/it]

  → saved full table with predictions through row 20694 to pert_checkpoint_up_to_20694.csv


In [7]:
from sklearn.metrics import classification_report, accuracy_score

# Define valid classes
valid_labels = ["group_2", "group_1", "no_bias"]

# Clean up casing and spacing
df['bias_type_merged'] = df['bias_type_merged'].astype(str).str.strip().str.lower()
df['predicted_bias'] = df['predicted_bias'].astype(str).str.strip().str.lower()

# Filter to rows with valid labels only
df_clean = df[
    df['predicted_bias'].isin(valid_labels) &
    df['bias_type_merged'].isin(valid_labels)
].copy()

# Accuracy
df_clean['correct'] = df_clean['bias_type_merged'] == df_clean['predicted_bias']
accuracy = df_clean['correct'].mean()
print(f"\nAccuracy: {accuracy * 100:.2f}%")

# Precision, Recall, F1
print("\nClassification Report:")
print(classification_report(
    df_clean['bias_type_merged'],
    df_clean['predicted_bias'],
    labels=valid_labels,
    digits=3
))




Accuracy: 43.54%

Classification Report:
              precision    recall  f1-score   support

     group_2      0.637     0.235     0.344      6969
     group_1      0.670     0.183     0.288      7339
     no_bias      0.374     0.943     0.536      6386

    accuracy                          0.435     20694
   macro avg      0.560     0.454     0.389     20694
weighted avg      0.568     0.435     0.383     20694



In [8]:
print(df["predicted_bias"].unique())
print(df["predicted_bias"].value_counts())

['no_bias' 'group_1' 'group_2']
predicted_bias
no_bias    16111
group_2     2577
group_1     2006
Name: count, dtype: int64
